In [1]:
import pandas as pd
import geopandas

In [ ]:
# world_map = geopandas.read_file('filtered_countries.geojson')
# world_map.loc[world_map['name'].str.contains('France')].iloc[0,2]

In [13]:
countries = {
    'BR': {'country': 'BR', 'country_name': 'Brazil', 'latitude': -15.749997, 'longitude': -51.92528}, 
    'CA': {'country': 'CA', 'country_name': 'Canada', 'latitude': 56.130366, 'longitude': -106.346771},
    'DE': {'country': 'DE', 'country_name': 'Germany', 'latitude': 51.165691, 'longitude': 10.451526},
    'FR': {'country': 'FR', 'country_name': 'France', 'latitude': 46.227638, 'longitude': 2.213749},
    'GB': {'country': 'GB', 'country_name': 'UK', 'latitude': 55.378051, 'longitude': -3.435973},
    'IN': {'country': 'IN', 'country_name': 'India', 'latitude': 20.593684, 'longitude': 78.96288},
    'JP': {'country': 'JP', 'country_name': 'Japan', 'latitude': 36.204824, 'longitude': 138.252924},
    'KR': {'country': 'KR', 'country_name': 'South Korea', 'latitude': 35.907757, 'longitude': 127.766922},
    'MX': {'country': 'MX', 'country_name': 'Mexico', 'latitude': 23.634501, 'longitude': -102.552784},
    'RU': {'country': 'RU', 'country_name': 'Russia', 'latitude': 61.52401, 'longitude': 105.318756},
    'US': {'country': 'US', 'country_name': 'USA', 'latitude': 37.09024, 'longitude': -95.712891}}

countries_df = pd.DataFrame(countries.values())

# countries_df.to_csv('countries.csv', index = False)

# countries_df

df = pd.read_csv('df_youtube_raw.csv')

df = df.drop(columns = [df.columns[0], 'publish_time', 'video_id', 'thumbnail_link', 
                         'ratings_disabled', 'comments_disabled', 'description'])

df = df.merge(countries_df, on='country')
df = df.drop(columns = ['country'])

df['views'] = pd.to_numeric(df['views'], downcast='integer')
df['likes'] = pd.to_numeric(df['likes'], downcast='integer')
df['dislikes'] = pd.to_numeric(df['dislikes'], downcast='integer')
df['comment_count'] = pd.to_numeric(df['comment_count'], downcast='integer')
# df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%d %H:%M:%S')
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%Y-%m-%d')

# df = df.loc[df['trending_date'] > '2020-01-01']
df = df.dropna()

def transformTags(x):
#     return "|".join(x.strip().replace('"', '').replace('[None]', '').replace('[none]', '').lower().split('|'))
    return x.strip().replace('"', '').replace('[None]', '').replace('[none]', '').lower().split('|')

df['tags'] = df['tags'].apply(transformTags)

In [14]:
bubble_map_data = df.groupby(['country_name', 'latitude', 'longitude']).agg({'title': 'count'}).rename(columns = {'title': 'value'})

bubble_map_data.to_csv('bubble_map_data.csv', index = True)

pd.read_csv('bubble_map_data.csv')

,country_name,latitude,longitude,value
0,Brazil,-15.749997,-51.925280,46887
1,Canada,56.130366,-106.346771,87612
2,France,46.227638,2.213749,87424
3,Germany,51.165691,10.451526,87357
4,India,20.593684,78.962880,77034
5,Japan,36.204824,138.252924,61642
6,Mexico,23.634501,-102.552784,86824
7,Russia,61.524010,105.318756,84877
8,South Korea,35.907757,127.766922,75687
9,UK,55.378051,-3.435973,85501


In [15]:
doughnut = df.groupby(['country_name', 'category_title']).agg({'title': 'count'}).rename(columns = {'title': 'count'}).reset_index()

doughnut = doughnut.merge(bubble_map_data, on = 'country_name').rename(columns = {'value' : 'total'})

doughnut['value'] = round(doughnut['count'] / doughnut['total'] * 100, 1)

doughnut.index = doughnut['country_name']

doughnut = doughnut.drop(columns = ['country_name', 'count', 'total'])

doughnut_temp = df.groupby(['category_title']).agg({'title': 'count'}).rename(columns = {'title': 'value'}).reset_index()

doughnut_temp['value'] = round(doughnut_temp['value'] / doughnut_temp.sum()['value'] * 100, 1)

doughnut_temp.index = ['Overall'] * doughnut_temp.shape[0]

doughnut_temp.index.name = 'country_name'

doughnut = doughnut.append(doughnut_temp)

main_list = []

for i in doughnut.index.unique():    
    as_list = doughnut.loc[i]['category_title'].tolist()
    
    for j in range(len(as_list)):        
        if doughnut.loc[i][doughnut.loc[i].category_title.str.contains(as_list[j])]['value'][0] < 3.5:
            as_list[j] = 'Others'

    main_list += as_list


doughnut['category_title'] = main_list

doughnut = doughnut.groupby(['country_name', 'category_title']).sum().round(1).sort_values(['country_name', 'value'], ascending = [True, False])

doughnut.to_csv('doughnut.csv', index = True)

pd.read_csv('doughnut.csv')

doughnut.head(20)

value
country_name category_title             
Brazil       Music                  25.8
             Entertainment          25.5
             Sports                 15.3
             People & Blogs         12.0
             Others                  9.5
             Gaming                  7.1
             Comedy                  4.8
Canada       Entertainment          26.7
             Music                  12.7
             Sports                  9.9
             People & Blogs          9.3
             Comedy                  8.0
             Gaming                  7.8
             News & Politics         6.8
             Others                  5.8
             Howto & Style           4.8
             Film & Animation        4.1
             Science & Technology    3.9
France       Entertainment          23.9
             Music                  17.4

In [178]:
def flatList(x):
    return [item for sublist in x for item in sublist if len(item) > 1]

wordcloud = pd.DataFrame(df.groupby('country_name')['tags'].apply(list))
wordcloud = pd.DataFrame(wordcloud['tags'].apply(flatList))
wordcloud.loc['Overall'] = [[item for list in wordcloud['tags'].to_list() for item in list]]

def topNWords(x, n = 80):
    return pd.DataFrame(x[0]).value_counts().sort_values(ascending = False).head(n)

wordcloud = pd.DataFrame(wordcloud.groupby('country_name')['tags'].apply(topNWords)).reset_index().rename(columns = {0: 'words', 'tags': 'count'})
wordcloud.index = wordcloud.country_name
wordcloud[['words', 'count']].to_csv('wordcloud.csv', index = True)
pd.read_csv('wordcloud.csv')

# wordcloud = wordcloud[['words', 'count']].groupby('country_name')['words'].apply(list).apply(lambda x: "|".join(x))

,country_name,words,count
0,Brazil,futebol,2442
1,Brazil,vlog,1904
2,Brazil,flamengo,1597
3,Brazil,humor,1536
4,Brazil,ao vivo,1485
...,...,...,...
955,USA,film,859
956,USA,animated,859
957,USA,snl,857
958,USA,love,850


In [179]:
df

,title,channel_title,trending_date,tags,views,likes,dislikes,comment_count,category_title,country_name,latitude,longitude
0,Ferrugem - Casa do Amor (Clipe Oficial),Ferrugem Oficial,2020-11-14,[],505194,16042,108,570,Film & Animation,Brazil,-15.749997,-51.925280
1,8 COISAS QUE ROLAM NA AULA ONLINE (Animação),aNIMA Lima,2020-11-26,"[animalima, animação, desenho, animado, flash,...",100750,18986,134,1511,Film & Animation,Brazil,-15.749997,-51.925280
2,Disney and Pixar's Luca | Teaser Trailer,Pixar,2021-03-03,"[pixar, disney, disney pixar, pixar movie, ani...",4855626,105694,2414,8649,Film & Animation,Brazil,-15.749997,-51.925280
3,Mulher-Maravilha 1984 - Trailer Oficial Principal,Warner Bros. Pictures Brasil,2020-08-25,"[mulher-maravilha, mulher-maravilha 1984, wond...",547752,25299,334,1042,Film & Animation,Brazil,-15.749997,-51.925280
4,The Witch - Animation vs. Minecraft Shorts Ep 21,Alan Becker,2021-02-09,[],7146081,372395,4906,34566,Film & Animation,Brazil,-15.749997,-51.925280
...,...,...,...,...,...,...,...,...,...,...,...,...
868650,Game of Zones - S5:E5: The Isle of Van Gundy,Bleacher Report,2018-05-20,"[bleacher report, br, nba, stan van gundy, jef...",1151096,21196,573,1599,Shows,USA,37.090240,-95.712891
868651,Apple HomePod: Everything to know before you b...,CNET,2018-01-27,"[apple, homepod, smart home, google home, amaz...",52255,932,85,268,Shows,USA,37.090240,-95.712891
868652,Apple HomePod: Everything to know before you b...,CNET,2018-01-30,"[apple, homepod, smart home, google home, amaz...",72607,1130,113,264,Shows,USA,37.090240,-95.712891
868653,Apple HomePod: Everything to know before you b...,CNET,2018-01-29,"[apple, homepod, smart home, google home, amaz...",66284,1068,105,263,Shows,USA,37.090240,-95.712891


In [198]:
engagement_data = df.groupby(['country_name', 'category_title']).agg({'comment_count': 'sum', 'likes': 'sum', 'views': 'sum'}).rename(columns = {'comment_count': 'comments'})

engagement_data.to_csv('engagement_data.csv', index = True)

pd.read_csv('engagement_data.csv')

,country_name,category_title,comments,likes,views
0,Brazil,Autos & Vehicles,775713,1.462018e+07,1.321137e+08
1,Brazil,Comedy,7398436,2.054700e+08,2.051216e+09
2,Brazil,Education,867859,1.689870e+07,1.791079e+08
3,Brazil,Entertainment,72072228,1.215684e+09,1.211587e+10
4,Brazil,Film & Animation,2938125,4.280812e+07,8.177259e+08
...,...,...,...,...,...
167,USA,Pets & Animals,4035026,3.592048e+07,1.125883e+09
168,USA,Science & Technology,27425873,2.828296e+08,9.070718e+09
169,USA,Shows,95117,1.082639e+06,5.150106e+07
170,USA,Sports,35584477,3.278868e+08,1.515717e+10


In [ ]:
# def expand(x):
#     return x

# arr = list(df.loc[df['country_name'] == 'Germany']['tags'].apply(expand))
# all_tags = [item.lower() for a in arr for item in a]

# df_alltags = pd.DataFrame(all_tags).rename(columns = {0: 'tags'})
# all_tags_count = pd.DataFrame(df_alltags.value_counts())

In [ ]:
# all_tags_count.head(40)

# all_tags_count.filter(like = 'none', axis = 0)

# df.filter(like = 'index to keep', axis=0)